In [2]:
from lxml import etree

In [3]:
parser = etree.HTMLParser()
root = './pages/pag'

In [4]:
from unicodedata import normalize

def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [5]:
# run into the element recursively getting the text
def get_element_text(elem):
    if len(elem) == 0:
        if elem != []:
            if elem.tag == "br":
                return "\n"
            elif elem.text == None:
                return ""
            else:
                return elem.text
        else:
            return ""
    else:
        if elem[0].tag == 'br':
            return "\n" + ("" if elem[0].tail == None else elem[0].tail.strip()) + get_element_text(elem[1:])
        if elem[0].tail != None and elem[0].tail.strip() != "":
            return (elem.text if isinstance(elem, etree._Element) and elem.text != None else "") + ("" if elem[0].text == None else elem[0].text) + ("" if len(elem[0]) == 0 else get_element_text(elem[0][:])) + elem[0].tail + get_element_text(elem[1:])
        else:
            return (elem.text if isinstance(elem, etree._Element) and elem.text != None else "") +  ("" if elem[0].text == None else elem[0].text) + ("" if len(elem[0]) == 0 else get_element_text(elem[0][:])) + get_element_text(elem[1:])

### El País

In [5]:
xpath_movie_detail1='//*[@id="sumario_1|despiece"]/div/div/p'
xpath_movie_detail2='//*[@id="sumario_2|despiece"]/div/div/p'
xpath_movie_title='//*[@id="sumario_1|despiece"]/div/header/h4'

filmes =[]
for i in range (1,11):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    r=tree.xpath(xpath_movie_title)
    if r != []:
        filme['nome'] = get_element_text(r)
    r=tree.xpath(xpath_movie_detail2)
    if len(r) == 0:
        r=tree.xpath(xpath_movie_detail1)

    if len(r) > 0:
        for x, item in enumerate(r):
            texto = get_element_text(item)
            if x == 0 and "nome" not in filme:
                filme["nome"] = texto
                continue
            
            try:
                indice = texto.index(":")
            except ValueError:
                indice = -1
            
            if(indice > -1):
                categoria = texto[:indice].lower()
                conteudo = texto[indice+1:].strip()
                
                if categoria == "direção":
                    filme["direcao"] = conteudo
                elif categoria == "gênero":
                    filme["genero"] = conteudo
                elif categoria == "elenco" or categoria == "intérpretes":
                    filme["elenco"] = conteudo
                elif categoria == "duração":
                    filme["duracao"] = conteudo
                elif "outro" in filme:
                    filme["outro"] += texto +";"
                else:
                    filme["outro"] = texto +";"
            elif "outro" in filme:
                filme["outro"] += texto +";"
            else:
                filme["outro"] = texto +";"
        
        filmes.append(filme)
filmes

[{'nome': 'HOMEM-FORMIGA E A VESPA',
  'direcao': 'Peyton Reed.',
  'elenco': 'Paul Rudd, Evangeline Lilly, Michael Douglas, Michelle Pfeiffer.',
  'genero': 'ação. EUA, 2018.',
  'duracao': '118 minutos.'},
 {'nome': 'NICO, 1988',
  'direcao': 'Susanna Nicchiarelli.',
  'elenco': 'Tryne Dyrholm, John Gordon Sinclair, Anamaria Marinca, Sandor Funtek.',
  'genero': 'musical. Itália, 2017.',
  'duracao': '93 minutos.'},
 {'nome': 'HOTEL TRANSILVÂNIA 3:\xa0 FÉRIAS MONSTRUOSAS',
  'direcao': 'Genndy Tartakovsky.',
  'outro': 'Animação.;',
  'genero': 'comédia. Estados Unidos, 2018.',
  'duracao': '97 minutos.'},
 {'nome': 'MAMMA MIA! LÁ VAMOS NÓS DE NOVO',
  'direcao': 'Ol Parker.',
  'elenco': 'Lily James, Amanda Seyfried, Christine Baranski, Pierce Brosnan.',
  'genero': 'musical. Reino Unido, 2018.',
  'duracao': '114 minutos.'},
 {'nome': 'MISSÃO IMPOSSÍVEL − EFEITO FALLOUT',
  'direcao': 'Christopher McQuarrie.',
  'elenco': 'Tom Cruise, Sean Harris, Henry Cavill, Rebecca Ferguson.',


In [6]:
elpais = [{'nome': 'HOMEM-FORMIGA E A VESPA',  'direcao': 'Peyton Reed.',  'elenco': 'Paul Rudd, Evangeline Lilly, Michael Douglas, Michelle Pfeiffer.',  'genero': 'ação. EUA, 2018.',  'duracao': '118 minutos.'}, {'nome': 'NICO, 1988',  'direcao': 'Susanna Nicchiarelli.',  'elenco': 'Tryne Dyrholm, John Gordon Sinclair, Anamaria Marinca, Sandor Funtek.',  'genero': 'musical. Itália, 2017.',  'duracao': '93 minutos.'}, {'nome': 'HOTEL TRANSILVÂNIA 3:\xa0 FÉRIAS MONSTRUOSAS',  'direcao': 'Genndy Tartakovsky.',  'outro': 'Animação.;',  'genero': 'comédia. Estados Unidos, 2018.',  'duracao': '97 minutos.'}, {'nome': 'MAMMA MIA! LÁ VAMOS NÓS DE NOVO',  'direcao': 'Ol Parker.',  'elenco': 'Lily James, Amanda Seyfried, Christine Baranski, Pierce Brosnan.',  'genero': 'musical. Reino Unido, 2018.',  'duracao': '114 minutos.'}, {'nome': 'MISSÃO IMPOSSÍVEL − EFEITO FALLOUT',  'direcao': 'Christopher McQuarrie.',  'elenco': 'Tom Cruise, Sean Harris, Henry Cavill, Rebecca Ferguson.',  'genero': 'thriller.',  'outro': 'Estados Unidos, 2018;',  'duracao': '147 minutos.'}, {'nome': 'Escobar – A Traição',  'outro': '(Loving Pablo);',  'direcao': 'Fernando León de Aranoa.',  'elenco': 'Javier Bardem, Penélope Cruz, Peter Sarsgaard.',  'genero': '‘biopic’. Espanha, 2017.',  'duracao': '123 minutos.'}, {'nome': '‘Pantera Negra’',  'direcao': 'Ryan Coogler.',  'elenco': 'Chadwick Boseman, Michael B. Jordan, Lupita Nyong’o, Danai Gurira.',  'genero': 'ficção científica. Estados Unidos, 2018.',  'duracao': '134 minutos.',  'outro': 'Estreia no Brasil: 15 de fevereiro.;'}, {'nome': 'TRAMA FANTASMA',  'direcao': 'Paul Thomas Anderson.',  'elenco': 'Daniel Day-Lewis, Lesley Manville, Vicky Krieps, Richard Graham.',  'genero': 'drama. EUA, 2017.',  'duracao': '130 minutos.'}, {'nome': 'TULLY',  'direcao': 'Jason Reitman.',  'elenco': 'Charlize Theron, Mackenzie Davis, Rum Livingston, Mark Duplass.',  'genero': 'drama. EUA, 2018.',  'duracao': '95 minutos.'}, {'nome': 'SUBMERSÃO',  'direcao': 'Wim Wenders.',  'elenco': 'James McAvoy, Alicia Vikander, Alexander Siddig, Celyn Jones.',  'genero': 'drama. Alemanha, 2017.',  'duracao': '111 minutos.'}]
filmes == elpais

True

### Omelete

In [7]:
xpath_movie_title = '/html/body/main/div[1]/section[1]/div[3]/div/div/article/div[3]/div[1]/div[1]/article/a/div[1]/div[2]/div[1]'
xpath_movie_subtitle = '/html/body/main/div[1]/section[1]/div[3]/div/div/article/div[3]/div[1]/div[1]/article/a/div[1]/div[2]/div[2]'
xpath_movie_details1 = '/html/body/main/div[1]/section[1]/div[3]/div/div/article/div[3]/div[1]/div[1]/article/a/div[2]/div/div[2]/div[1]/p'
xpath_movie_details2 = '/html/body/main/div[1]/section[1]/div[3]/div/div/article/div[3]/div[1]/div[1]/article/a/div[2]/div/div[2]/div[2]/p'

filmes=[]
for i in range (11,21):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    r=tree.xpath(xpath_movie_title)
    filme['nome'] = get_element_text(r)
    r=tree.xpath(xpath_movie_details1)+tree.xpath(xpath_movie_details2)
    for item in r:
        categoria = remover_acentos(item.text.replace("Ã§Ã£","çã").strip().lower()).replace(":","")
        filme[categoria] = item[0].text.strip()
    filmes.append(filme)
filmes

[{'nome': 'Hotel Artemis',
  'ano': '2018',
  'classificacao': '16 anos',
  'duracao': '93 min',
  'direcao': 'Drew Pearce',
  'roteiro': 'Drew Pearce',
  'elenco': 'Zachary Quinto, Jenny Slate, Jeff Goldblum, Sofia Boutella, Sterling K. Brown, Jodie Foster'},
 {'nome': 'Nasce Uma Estrela',
  'ano': '2018',
  'classificacao': '-1 anos',
  'direcao': 'Bradley Cooper',
  'roteiro': 'Bradley Cooper, Will Fetters, Irene Mecchi, Christopher Wilkinson, Stephen J. Rivele',
  'elenco': 'Bradley Cooper, Lady Gaga, Sam Elliott'},
 {'nome': 'Red Joan',
  'classificacao': '-1 anos',
  'direcao': 'Trevor Nunn',
  'elenco': 'Tom Hughes, Stephen Campbell Moore, Sophie Cookson, Judi Dench'},
 {'nome': 'Destroyer',
  'ano': '2018',
  'classificacao': '-1 anos',
  'direcao': 'Karyn Kusama',
  'roteiro': 'Matt Manfredi, Phil Hay',
  'elenco': 'Toby Kebbell, Tatiana Maslany, Nicole Kidman'},
 {'nome': 'O Predador (2018)',
  'ano': '2018',
  'classificacao': '-1 anos',
  'direcao': 'Shane Black',
  'elenco

In [ ]:
omelete = [{'nome': 'Hotel Artemis',  'ano': '2018',  'classificacao': '16 anos',  'duracao': '93 min',  'direcao': 'Drew Pearce',  'roteiro': 'Drew Pearce',  'elenco': 'Zachary Quinto, Jenny Slate, Jeff Goldblum, Sofia Boutella, Sterling K. Brown, Jodie Foster'}, {'nome': 'Nasce Uma Estrela',  'ano': '2018',  'classificacao': '-1 anos',  'direcao': 'Bradley Cooper',  'roteiro': 'Bradley Cooper, Will Fetters, Irene Mecchi, Christopher Wilkinson, Stephen J. Rivele',  'elenco': 'Bradley Cooper, Lady Gaga, Sam Elliott'}, {'nome': 'Red Joan',  'classificacao': '-1 anos',  'direcao': 'Trevor Nunn',  'elenco': 'Tom Hughes, Stephen Campbell Moore, Sophie Cookson, Judi Dench'}, {'nome': 'Destroyer',  'ano': '2018',  'classificacao': '-1 anos',  'direcao': 'Karyn Kusama',  'roteiro': 'Matt Manfredi, Phil Hay',  'elenco': 'Toby Kebbell, Tatiana Maslany, Nicole Kidman'}, {'nome': 'O Predador (2018)',  'ano': '2018',  'classificacao': '-1 anos',  'direcao': 'Shane Black',  'elenco': 'Olivia Munn, Thomas Jane, Boyd Holbrook, Sterling K. Brown, Keegan-Michael Key'}, {'nome': 'Mid90s',  'ano': '2018',  'classificacao': '-1 anos',  'duracao': '1h24 min',  'direcao': 'Jonah Hill',  'roteiro': 'Jonah Hill',  'elenco': 'Katherine Waterston, Lucas Hedges, Sunny Suljic'}, {'nome': 'A Primeira Noite de Crime',  'ano': '2018',  'classificacao': '-1 anos',  'direcao': 'Gerard McMurray',  'roteiro': 'James DeMonaco',  'elenco': "John P. McGinty, Cloé Xhauflaire, Jessica Bell, Geoff Schuppert, Qurrat Ann Kadwani, Christian Robinson, David Breda, Lex Scott Davis, Y'lan Noel"}, {'nome': 'O MistÃ©rio do RelÃ³gio na Parede',  'ano': '2018',  'classificacao': '10 anos',  'duracao': '105 min',  'direcao': 'Eli Roth',  'roteiro': 'Eric Kripke',  'elenco': 'Kyle MacLachlan, Owen Vaccaro, Cate Blanchett, Jack Black'}, {'nome': 'American Woman',  'ano': '2018',  'classificacao': '-1 anos',  'direcao': 'Jake Scott',  'roteiro': 'Brad Ingelsby',  'elenco': 'Macon Blair, Will Sasso, Amy Madigan, Sienna Miller, Aaron Paul, Christina Hendricks'}, {'nome': 'Buscando...',  'ano': '2018',  'classificacao': '-1 anos',  'direcao': 'Aneesh Chaganty',  'roteiro': 'Sev Ohanian, Aneesh Chaganty',  'elenco': 'Sara Sohn, John Cho'}]
filmes == omelete

### CineClick

In [8]:
titulo = '//*[@id="movie"]/div[2]/div/div/div[1]/div[2]/div[2]/h1'
ano_duracao='//*[@id="movie"]/div[2]/div/div/div[1]/div[2]/div[2]/p'
genero = '//*[@id="movie"]/div[2]/div/div/div[1]/div[2]/div[2]/div[2]/p[1]'
direcao='//*[@id="movie"]/div/ul'
details='//*[@id="movie"]/div/p'
elenco='//*[@id="movie"]/div/div/p'

filmes = []
for i in range (21,31):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    
    r=tree.xpath(titulo)
    filme["nome"]=r[0].text
    
    r=tree.xpath(ano_duracao)
    ad=get_element_text(r).strip().replace(" ","")
    try:
        filme["ano"]=ad[:ad.index(",")]
        filme["duracao"]=ad[ad.index(",")+1:]
    except ValueError:
        pass
    
    r=tree.xpath(genero)
    gen=get_element_text(r)
    try:
        filme["genero"]=gen[gen.index(":")+1:].strip()
    except ValueError:
        filme["genero"]=gen
        
    r=tree.xpath(direcao)
    filme["direcao"]=get_element_text(r[0]).strip()
    
    r=tree.xpath(details)
    for item in r:
        info = get_element_text(item)
        try:
            categoria=remover_acentos(info[:info.index(":")].lower())
            filme[categoria]=info[info.index(":")+1:]
        except ValueError:
            pass    
            
    r=tree.xpath(elenco)
    filme["elenco"]=get_element_text(r[0])
    filmes.append(filme)
filmes

[{'nome': 'O PREDADOR',
  'ano': '2018',
  'duracao': '101MIN.',
  'genero': 'Ação',
  'direcao': 'Shane Black',
  'roteiro': '     Fred Dekker, Jim Thomas, John Thomas, Shane Black',
  'producao': '     Blondel Aidoo, John Davis, Lawrence Gordon',
  'fotografia': '     Larry Fong',
  'trilha sonora': '     Henry Jackman',
  'estudio': '     Canada Film Capital, Dark Castle Entertainment, Davis Entertainment, TSG Entertainment, Twentieth Century Fox Film Corporation',
  'montador': '     Harry B. Miller III',
  'distribuidora': '     Fox Film',
  'elenco': '     Alfie Allen, Andrew Jenkins, Augusto Aguilera, Boyd Holbrook, Crystal Mudry, Edward James Olmos, Inka Malovic, Jacob Tremblay, Jake Busey, Keegan-Michael Key, Lochlyn Munro, Niall Matter, Olivia Munn, Patrick Sabongui, Paul Lazenby, RJ Fetherstonhaugh, Sterling K. Brown, Thomas Jane, Trevante Rhodes, Yvonne Strahovski'},
 {'nome': 'A FREIRA',
  'ano': '2018',
  'duracao': '96MIN.',
  'genero': 'Terror',
  'direcao': 'Corin Hard

In [9]:
cineclick=[{'nome': 'O PREDADOR',  'ano': '2018',  'duracao': '101MIN.',  'genero': 'Ação',  'direcao': 'Shane Black',  'roteiro': '     Fred Dekker, Jim Thomas, John Thomas, Shane Black',  'producao': '     Blondel Aidoo, John Davis, Lawrence Gordon',  'fotografia': '     Larry Fong',  'trilha sonora': '     Henry Jackman',  'estudio': '     Canada Film Capital, Dark Castle Entertainment, Davis Entertainment, TSG Entertainment, Twentieth Century Fox Film Corporation',  'montador': '     Harry B. Miller III',  'distribuidora': '     Fox Film',  'elenco': '     Alfie Allen, Andrew Jenkins, Augusto Aguilera, Boyd Holbrook, Crystal Mudry, Edward James Olmos, Inka Malovic, Jacob Tremblay, Jake Busey, Keegan-Michael Key, Lochlyn Munro, Niall Matter, Olivia Munn, Patrick Sabongui, Paul Lazenby, RJ Fetherstonhaugh, Sterling K. Brown, Thomas Jane, Trevante Rhodes, Yvonne Strahovski'}, {'nome': 'A FREIRA',  'ano': '2018',  'duracao': '96MIN.',  'genero': 'Terror',  'direcao': 'Corin Hardy',  'roteiro': '     Gary Dauberman, James Wan',  'producao': '     James Wan, Michael Clear, Peter Safran',  'fotografia': '     Maxime Alexandre',  'trilha sonora': '     Abel Korzeniowski',  'estudio': '     Atomic Monster, New Line Cinema, Safran Company, The',  'montador': '     Michel Aller',  'distribuidora': '     Warner',  'elenco': '     Ani Sava, August Maturo, Boiangiu Alma, Bonnie Aarons, Charlotte Hope, Demián Bichir, Emma Appleton, Flynn Hayward, Ingrid Bisu, Jamie Muscato, Jared Morgan, Jonas Bloquet, Jonny Coyne, Lourdes Nadres, Mark Steger, Natalie Creek, Samson Marraccino, Sandra Rosko, Sandra Teles, Scarlett Hicks, Simon Rhee, Taissa Farmiga'}, {'nome': 'BUSCANDO...',  'ano': '2018',  'duracao': '101MIN.',  'genero': 'Suspense',  'direcao': 'Aneesh Chaganty',  'roteiro': '     Aneesh Chaganty, Sev Ohanian',  'producao': '     Adam Sidman, Congyu E, Natalie Qasabian, Sev Ohanian, Timur Bekmambetov',  'fotografia': '     Juan Sebastian Baron',  'trilha sonora': '     Torin Borrowdale',  'estudio': '     Bazelevs Entertainment, Bazelevs Production',  'montador': '     Nicholas D. Johnson, Will Merrick',  'distribuidora': '     Sony Pictures',  'elenco': '     Alex Jayne Go, Ashley Edner, Benjamin J. Cain Jr., Brad Abrell, Courtney Lauren Cummings, Debra Messing, Erin Henriques, Gabriel D. Angell, Gage Biltoft, John Cho, Joseph Lee, Kyle Austin Brown, Lasaundra Gibson, Melissa Disney, Michelle La, Morgan Peter Brown, Rasha Goel, Reed Buck, Roy Abramsohn, Sara Sohn, Steven Michael Eich, Thomas Barbusca'}, {'nome': 'KIN',  'ano': '2018',  'duracao': '102MIN.',  'genero': 'Ação',  'direcao': 'Jonathan Baker, Josh Baker',  'roteiro': '     Daniel Casey, Jonathan Baker, Josh Baker',  'producao': '     Dan Cohen, David Gross, Jeff Arkuss, Jesse Shapira, Shawn Levy',  'fotografia': '     Larkin Seiple',  'trilha sonora': '     Mogwai',  'estudio': '     21 Laps Entertainment, Lionsgate, No Trace Camping',  'montador': '     Mark Day',  'distribuidora': '     Paris Filmes',  'elenco': '     Attila Sebesy, Bree Wasylenko, Carleigh Beverly, Carlos Pinder, Carrie Coon, Carson Manning, Dennis Quaid, Eli Ham, Gavin Fox, Ian Matthews, Ivan Sherry, Ivan Wanis-Ruiz, Jack Reynor, James Franco, Jonathan Cherry, Khalid Klein, Lily Gao, Myles Truitt, Romano Orzari, Shawn J. Hamilton, Stella Acquisto, Zoë Kravitz'}, {'nome': 'FERRUGEM',  'ano': '2018',  'duracao': '100MIN.',  'genero': 'Drama',  'direcao': 'Aly Muritiba',  'roteiro': '     Aly Muritiba, George Moura, Jessica Candal',  'producao': '     Ana Catarina, Antonio Júnior, Fernando Meirelles, Guel Arraes',  'fotografia': '     Rui Poças',  'trilha sonora': '     Fabian Oliver',  'estudio': '     Grafo Audiovisual',  'montador': '     João Menna Barreto',  'distribuidora': '     Olhar Distribuição',  'elenco': '     Clarissa Kiste, Dudah Azevedo, Enrique Diaz, Giovanni de Lorenzi, Igor Augustho, Pedro Inoue, Tifanny Dopke'}, {'nome': 'OS JOVENS TITÃS EM AÇÃO! NOS CINEMAS',  'ano': '2018',  'duracao': '93MIN.',  'genero': 'Animação',  'direcao': 'Aaron Horvath, Peter Rida Michail',  'roteiro': '     Aaron Horvath, Arnold Drake, Bob Brown, Bob Haney, Bruno Premiani, George Perez, Harry G. Peter, Marv Wolfman, Michael Jelenic',  'producao': '     Peggy Regan, Peter Rida Michail, Will Arnett',  'trilha sonora': '     Jared Faber',  'estudio': '     Cartoon Network, DC Entertainment, Warner Bros., Warner Bros. Animation',  'montador': '     Nick Kenway',  'distribuidora': '     Warner Bros',  'elenco': '     Dana Snyder, Greg Cipes, Halsey, Hynden Walch, James Corden, Jimmy Kimmel, Khary Payton, Kristen Bell, Lil Yachty, Nicolas Cage, Scott Menville, Tara Strong, Will Arnett'}, {'nome': 'SLENDER MAN - PESADELO SEM ROSTO',  'ano': '2018',  'duracao': '93MIN.',  'genero': 'Horror',  'direcao': 'Sylvain White',  'roteiro': '     David Birke, Victor Surge',  'producao': '     Andrea Ajemian, Bradley J. Fischer, James Vanderbilt, Robyn Meisinger, Sarah Snow, William Sherak',  'fotografia': '     Luca Del Puppo',  'trilha sonora': '     Brandon Campbell, Ramin Djawadi',  'estudio': '     Madhouse Entertainment, Mythology Entertainment',  'montador': '     Jake York',  'distribuidora': '     Sony',  'elenco': "     Adrian M. Mompoint, Alex Fitzalan, Angela Hope Smith, Annalise Basso, Damon D'Amico Jr, Javier Botet, Jaz Sinclair, Jeffrey Corazzini, Jessica Blank, Joey King, Julia Goldani Telles, Kallie Tabor, Kayla Caulfield, Kevin Chapman, Kris Sidberry, Michael Reilly Burke, Michael Tow, Miguel Nascimento, Oscar Wahlberg, Remington Keyes, Taylor Richardson"}, {'nome': 'MEU EX É UM ESPIÃO',  'ano': '2018',  'duracao': '116MIN.',  'genero': 'Ação',  'direcao': 'Susanna Fogel',  'roteiro': '     David Iserson, Susanna Fogel',  'producao': '     Brian Grazer, Erica Huggins',  'fotografia': '     Barry Peterson',  'trilha sonora': '     Tyler Bates',  'estudio': '     Imagine Entertainment, Lionsgate',  'montador': '     Jonathan Schwartz',  'distribuidora': '     Paris Filmes',  'elenco': '     Attila C. Arpa, Barnabás Réti, Dustin Demri-Burns, Fred Melamed, Genevieve McCarthy, Gillian Anderson, Hasan Minhaj, Ivanna Sakhno, Justin Theroux, Justine Wachsberger, Kate McKinnon, Kev Adams, Lolly Adefope, Maria Z. Wilson, Mila Kunis, Nick Vorsselman, Nick Wittman, Ólafur Darri Ólafsson, Roderick Hill, Ruby Kammer, Sam Heughan'}, {'nome': 'CHRISTOPHER ROBIN - UM REENCONTRO INESQUECÍVEL',  'ano': '2018',  'duracao': '104MIN.',  'genero': 'Animação',  'direcao': 'Marc Forster',  'roteiro': '     A.A. Milne, Alex Ross Perry, Allison Schroeder, Tom McCarthy',  'producao': '     Brigham Taylor, Steve Gaub',  'fotografia': '     Matthias Koenigswieser',  'trilha sonora': '     Geoff Zanelli, Jon Brion',  'estudio': '     Walt Disney Pictures',  'montador': '     Matt Chesse',  'distribuidora': '     Disney',  'elenco': "     Adrian Scarborough, Bern Collaco, Brad Garrett, Bronte Carmichael, Chris O'Dowd, Evie Wray, Ewan McGregor, Gintare Beinoraviciute, Hayley Atwell, Jim Cummings, Katy Carmichael, Mark Gatiss, Nick Mohammed, Oliver Ford Davies, Paul Chahidi, Peter Capaldi, Roger Ashton-Griffiths, Shola Adewusi, Sophie Okonedo, Toby Jones, Tristan Sturrock"}, {'nome': 'O PROTETOR 2',  'ano': '2018',  'duracao': '121MIN.',  'genero': 'Ação',  'direcao': 'Antoine Fuqua',  'roteiro': '     Michael Sloane, Richard Lindheim, Richard Wenk',  'producao': '     Alex Siskin, Denzel Washington, Jason Blumenthal, Mace Neufeld, Steve Tisch, Todd Black, Tony Eldridge',  'fotografia': '     Oliver Wood',  'trilha sonora': '     Harry Gregson-Williams',  'estudio': '     Columbia Pictures, Escape Artists, Fuqua Films, Lonetree Entertainment, Mace Neufeld Productions, Sony Pictures Entertainment (SPE)',  'montador': '     Conrad Buff IV',  'distribuidora': '     Sony Pictures',  'elenco': "     Abigail Marlowe, Alexa Giuffre, Alin Halajian, Andrei Arlovski, Annie Pisapia, Ashton Sanders, Bill Pullman, Caroline Day, Denzel Washington, Donald Cerrone, Enku Gubaie, Eric Ian, Heidi Garrow, Jay Hieron, Jonathan Scarfe, Karen Strong, Lexie Roth, Melissa Leo, Patty O'Neil, Pedro Pascal, Sakina Jaffrey, Tamara Hickey"}]
filmes==cineclick

True

### Cinema em Cena

In [10]:
xpath_details = '//*[@class="critica-elenco"]'
nome='/html/body/div/div/div/h1'
ano='/html/body/div/div/div/table/tbody/tr/td/text()'

# tree = etree.parse('./pages/pag31.html', parser)
# tree.xpath(ano)
filmes = []
for i in range(31,41):
    tree = etree.parse(root + str(i) + '.html', parser)
    
    categoria = ""
    detalhes = {}
    filme = {}
    filme["nome"] = tree.xpath(nome)[1].text.strip()
    
    info = tree.xpath(ano)
    if len(info) == 3:
        filme["ano"] = info[0]
        filme["distribuidora"] = info[2]
    elif len(info) == 2:
        filme["ano"] = info[0]
    
    for x in tree.xpath(xpath_details)[0]:
    #     if i.tag == "div" or i.tag == "br":
    #         flag = False
        if x.tag == "h3":
            categoria = remover_acentos(x.text.lower())
            detalhes[categoria] = []
    #         flag = True
        elif x.tag == "a":
            detalhes[categoria].append(x.text)
    for i in detalhes:
        filme[i] = ', '.join(detalhes[i])    
    filmes.append(filme)
    
filmes

[{'nome': 'Ilha dos Cachorros',
  'ano': '19/07/2018',
  'distribuidora': 'Fox',
  'direcao': 'Wes Anderson',
  'elenco': 'Bryan Cranston, Koyu Rankin, Edward Norton, Scarlett Johansson, Bob Balaban, Greta Gerwig, Akira Takayama, Yoko Ono, Akira Ito, Fisher Stevens, Liev Schreiber, Courtney B. Vance, Roman Coppola, Anjelica Huston, Ken Watanabe, F. Murray Abraham, Tilda Swinton, Frances McDormand, Harvey Keitel, Jeff Goldblum, Bill Murray',
  'roteiro': 'Wes Anderson',
  'producao': 'Wes Anderson, Jeremy Dawson, Scott Rudin, Steven Rales',
  'fotografia': 'Tristan Oliver',
  'musica': 'Alexandre Desplat',
  'montagem': 'Andrew Weisblum, Edward Bursch, Ralph Foster',
  'design de producao': 'Adam Stockhausen, Paul Harrod'},
 {'nome': 'Sicário: Dia do Soldado',
  'ano': '28/06/2018',
  'distribuidora': 'Sony',
  'direcao': 'Stefano Sollima',
  'elenco': 'Benicio Del Toro, Josh Brolin, Isabela Moner, Catherine Keener, Jeffrey Donovan, Matthew Modine, Shea Whigham, Manuel Garcia-Rulfo, Eli

In [11]:
cinemaemcena=[{'nome': 'Ilha dos Cachorros',  'ano': '19/07/2018',  'distribuidora': 'Fox',  'direcao': 'Wes Anderson',  'elenco': 'Bryan Cranston, Koyu Rankin, Edward Norton, Scarlett Johansson, Bob Balaban, Greta Gerwig, Akira Takayama, Yoko Ono, Akira Ito, Fisher Stevens, Liev Schreiber, Courtney B. Vance, Roman Coppola, Anjelica Huston, Ken Watanabe, F. Murray Abraham, Tilda Swinton, Frances McDormand, Harvey Keitel, Jeff Goldblum, Bill Murray',  'roteiro': 'Wes Anderson',  'producao': 'Wes Anderson, Jeremy Dawson, Scott Rudin, Steven Rales',  'fotografia': 'Tristan Oliver',  'musica': 'Alexandre Desplat',  'montagem': 'Andrew Weisblum, Edward Bursch, Ralph Foster',  'design de producao': 'Adam Stockhausen, Paul Harrod'}, {'nome': 'Sicário: Dia do Soldado',  'ano': '28/06/2018',  'distribuidora': 'Sony',  'direcao': 'Stefano Sollima',  'elenco': 'Benicio Del Toro, Josh Brolin, Isabela Moner, Catherine Keener, Jeffrey Donovan, Matthew Modine, Shea Whigham, Manuel Garcia-Rulfo, Elijah Rodriguez, Bruno Bichir, Raoul Trujillo',  'roteiro': 'Taylor Sheridan',  'producao': 'Basil Iwanyk, Thad Luckinbill, Trent Luckinbill, Edward L. McDonnell, Molly Smith',  'fotografia': 'Dariusz Wolski',  'musica': 'Hildur Guðnadóttir',  'montagem': 'Matthew Newman',  'design de producao': 'Kevin Kavanaugh',  'figurino': 'Deborah Lynn Scott'}, {'nome': 'Os Incríveis 2',  'ano': '28/06/2018',  'distribuidora': 'Disney',  'direcao': 'Brad Bird',  'elenco': 'Holly Hunter, Craig T. Nelson, Sarah Vowell, Samuel L. Jackson, Huck Milner, Catherine Keener, Bob Odenkirk, Brad Bird, Sophia Bush, Phil LaMarr, John Ratzenberger, Barry Bostwick, Adam Rodriguez, Kimberly Adair Clark, Paul Eiding, Jonathan Banks, Isabella Rossellini',  'roteiro': 'Brad Bird',  'producao': 'John Walker, Nicole Paradis Grindle',  'fotografia': 'Mahyar Abousaeedi, Erik Smitt',  'musica': 'Michael Giacchino',  'montagem': 'Stephen Schaffer',  'design de producao': 'Ralph Eggleston',  'figurino': 'Deanna Marsigliese'}, {'nome': 'As Boas Maneiras',  'ano': '07/06/2018',  'distribuidora': 'Imovision',  'direcao': 'Juliana Rojas',  'elenco': 'Isabél Zuaa, Marjorie Estiano, Cida Moreira, Andréa Marquee, Felipe Kenji, Nina Medeiros, Neusa Velasco, Gilda Nomacce, Naloana Lima, Miguel Lobo',  'roteiro': 'Marco Dutra, Juliana Rojas, Marco Dutra',  'producao': 'Sara Silveira, Clément Duboin, Frédéric Corvez',  'fotografia': 'Rui Poças',  'musica': 'Guilherme Garbato, Gustavo Garbato',  'montagem': 'Caetano Gotardo',  'design de producao': 'Fernando Zuccolotto',  'figurino': 'Kiki Orona'}, {'nome': 'Hereditário',  'ano': '21/06/2018',  'distribuidora': 'Diamond Films',  'direcao': 'Ari Aster',  'elenco': 'Toni Collette, Gabriel Byrne, Alex Wolff, Milly Shapiro, Mallory Bechtel, David Stanley, Ann Dowd',  'roteiro': 'Ari Aster',  'producao': 'Lars Knudsen, Kevin Scott Frakes, Buddy Patrick',  'fotografia': 'Pawel Pogorzelski',  'musica': 'Colin Stetson',  'montagem': 'Jennifer Lame, Lucian Johnston',  'design de producao': 'Grace Yun',  'figurino': 'Olga Mill'}, {'nome': 'O Amante Duplo',  'ano': '21/06/2018',  'distribuidora': 'California Filmes',  'direcao': 'François Ozon',  'elenco': 'Marine Vacth, Jérémie Renier, Jacqueline Bisset, Myriam Boyer, Dominique Reymond',  'roteiro': 'François Ozon, Philippe Piazzo',  'producao': 'Eric Altmayer, Nicolas Altmayer',  'fotografia': 'Manuel Dacosse',  'musica': 'Philippe Rombi',  'montagem': 'Laure Gardette',  'design de producao': 'Sylvie Olivé',  'figurino': 'Pascaline Chavanne'}, {'nome': 'Jurassic World: Reino Ameaçado',  'ano': '21/06/2018',  'distribuidora': 'Universal',  'direcao': 'J.A. Bayona',  'elenco': 'Chris Pratt, Bryce Dallas Howard, Ted Levine, Toby Jones, James Cromwell, BD Wong, Rafe Spall, Daniella Pineda, Justice Smith, Isabella Sermon, Geraldine Chaplin, Jeff Goldblum',  'roteiro': 'Colin Trevorrow, Derek Connolly',  'producao': 'Patrick Crowley, Belén Atienza, Frank Marshall',  'fotografia': 'Oscar Faura',  'musica': 'Michael Giacchino',  'montagem': 'Bernat Vilaplana',  'design de producao': 'Andy Nicholson',  'figurino': 'Sammy Sheldon'}, {'nome': 'Dovlatov',  'ano': '14/06/2018',  'distribuidora': 'Imovision',  'direcao': 'Aleksey German',  'elenco': 'Milan Maric, Danila Kozlovsky, Helena Sujecka, Artur Beschastny, Elena Lyadova, Anton Shagin, Svetlana Khodchenkova, Piotr Gasowski, Eva Gerr, Hanna Sleszynska',  'roteiro': 'Aleksey German, Yulia Tupikina',  'producao': 'Dariusz Jablonski, Violetta Kaminska, Miroslav Mogorovich, Isabella Wojcik',  'fotografia': 'Lukasz Zal',  'montagem': 'Sergey Ivanov, Daria Gladysheva',  'design de producao': 'Elena Okopnaya',  'figurino': 'Elena Okopnaya'}, {'nome': 'In the Game',  'ano': 'Unknown',  'direcao': 'Maria Finitzo',  'roteiro': 'Maria Finitzo',  'producao': 'Maria Finitzo, Mary Morrissette',  'fotografia': 'Peter Gilbert',  'montagem': 'Katerina Simic'}, {'nome': 'Você Nunca Esteve Realmente Aqui',  'ano': '09/08/2018',  'distribuidora': 'Supo Mungan Films',  'direcao': 'Lynne Ramsay',  'elenco': 'Joaquin Phoenix, Ekaterina Samsonov, Alex Manette, John Doman, Dante Pereira-Olson, Judith Roberts, Alessandro Nivola',  'roteiro': 'Lynne Ramsay',  'producao': "Pascal Caucheteux, Rebecca O'Brien, Lynne Ramsay, Rosa Attab, James Wilson",  'fotografia': 'Thomas Townend',  'musica': 'Jonny Greenwood',  'montagem': 'Joe Bini',  'design de producao': 'Tim Grimes',  'figurino': 'Malgosia Turzanska'}]
filmes == cinemaemcena

True

### Jornal da Paraíba

In [12]:
xpath_details = '//*[@id="noticias"]/div[1]/article/div[3]/ul/li'
xpath_title = '//*[@id="noticias"]/div[1]/article/header/h3'

filmes = []
for i in range(41,51):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    name = tree.xpath(xpath_title)
    elems = tree.xpath(xpath_details)
    
    filme["nome"]=get_element_text(name[0])
    for item in elems:
        i = get_element_text(item)
        try:
            categoria = remover_acentos(i[:i.index(":")].lower())
            filme[categoria] = i[i.index(":")+1:].strip()
        except ValueError:
            pass
    filmes.append(filme)
filmes

[{'nome': 'O Mistério do Relógio na Parede',
  'genero': 'Fantasia, Mistério, Terror',
  'censura': '10 - Não recomendado para menores de dez anos',
  'duracao': '104 min',
  'diretor': 'Eli Roth',
  'elenco': 'Cate Blanchett, Colleen Camp, Jack Black, Kyle MacLachlan, Renée Elise Goldsberry'},
 {'nome': 'Buscando…',
  'genero': 'Drama, Mistério, Thriller',
  'censura': '14 - Não recomendado para menores de catorze anos',
  'duracao': '102 min',
  'diretor': 'Aneesh Chaganty',
  'elenco': 'Brad Abrell, Debra Messing, Gabriel D. Angell, John Cho, Joseph Lee, Michelle La, Roy Abramsohn, Sara Sohn'},
 {'nome': 'Benzinho',
  'genero': 'Drama',
  'censura': '12 - Não recomendado para menores de doze anos',
  'duracao': '97 min',
  'diretor': 'Gustavo Pizzi',
  'elenco': 'Adriana Esteves, Arthur Teles Pizzi, César Troncoso, Francisco Teles Pizzi, Karine Teles, Konstantinos Sarris, Luan Teles, Mateus Solano, Otávio Müller, Vicente Demori'},
 {'nome': '22 Milhas',
  'genero': 'Ação',
  'censur

In [13]:
jornal = [{'nome': 'O Mistério do Relógio na Parede',  'genero': 'Fantasia, Mistério, Terror',  'censura': '10 - Não recomendado para menores de dez anos',  'duracao': '104 min',  'diretor': 'Eli Roth',  'elenco': 'Cate Blanchett, Colleen Camp, Jack Black, Kyle MacLachlan, Renée Elise Goldsberry'}, {'nome': 'Buscando…',  'genero': 'Drama, Mistério, Thriller',  'censura': '14 - Não recomendado para menores de catorze anos',  'duracao': '102 min',  'diretor': 'Aneesh Chaganty',  'elenco': 'Brad Abrell, Debra Messing, Gabriel D. Angell, John Cho, Joseph Lee, Michelle La, Roy Abramsohn, Sara Sohn'}, {'nome': 'Benzinho',  'genero': 'Drama',  'censura': '12 - Não recomendado para menores de doze anos',  'duracao': '97 min',  'diretor': 'Gustavo Pizzi',  'elenco': 'Adriana Esteves, Arthur Teles Pizzi, César Troncoso, Francisco Teles Pizzi, Karine Teles, Konstantinos Sarris, Luan Teles, Mateus Solano, Otávio Müller, Vicente Demori'}, {'nome': '22 Milhas',  'genero': 'Ação',  'censura': '14 - Não recomendado para menores de catorze anos',  'duracao': '94 min',  'diretor': 'Peter Berg',  'elenco': 'Iko Uwais, John Malkovich, Lauren Cohan, Mark Wahlberg, Ronda Rousey, Terry Kinney'}, {'nome': 'Os Jovens Titãs em Ação! Nos Cinemas',  'genero': 'Ação, Animação, Comédia',  'diretor': 'Aaron Horvath, Peter Rida Michail',  'elenco': 'Greg Cipes, Hynden Walch, Khary Payton, Kristen Bell, Nicolas Cage, Scott Menville, Tara Strong, Will Arnett'}, {'nome': 'O Candidato Honesto 2',  'genero': 'Comédia',  'censura': '14 - Não recomendado para menores de catorze anos',  'duracao': '108 min',  'diretor': 'Roberto Santucci',  'elenco': 'Flávia Garrafa, Leandro Hassum, Rosane Mulholland'}, {'nome': 'O Predador – A Caçada Evoluiu',  'genero': 'Ação, Aventura, Terror',  'censura': '18 - Não recomendado para menores de dezoito anos',  'duracao': '107 min',  'diretor': 'Shane Black',  'elenco': 'Boyd Holbrook, Jacob Tremblay, Keegan-Michael Key, Olivia Munn, Sterling K. Brown, Trevante Rhodes'}, {'nome': 'Crô em Família',  'genero': 'Comédia',  'censura': '12 - Não recomendado para menores de doze anos',  'duracao': '87 min',  'diretor': 'Cininha de Paula',  'elenco': 'Arlete Salles, Fabiana Karla, Marcelo Serrado, Marcos Caruso, Marcus Majella, Mateus Solano, Tonico Pereira'}, {'nome': 'Alfa',  'genero': 'Ação, Aventura, Drama',  'censura': '10 - Não recomendado para menores de dez anos',  'duracao': '97 min',  'diretor': 'Albert Hughes',  'elenco': 'Jens Hultén, Jóhannes Haukur Jóhannesson, Kodi Smit-McPhee, Leonor Varela, Mercedes de la Zerda, Natassia Malthe'}, {'nome': 'Ilha dos Cachorros',  'genero': 'Animação, Aventura, Comédia',  'censura': '12 - Não recomendado para menores de doze anos',  'duracao': '101 min',  'diretor': 'Wes Anderson',  'elenco': 'Bill Murray, Bryan Cranston, Edward Norton, Frances McDormand, Greta Gerwig, Jeff Goldblum, Scarlett Johansson, Tilda Swinton'}]
filmes == jornal

True

### Cinema com Rapadura

In [14]:
xpath_details = '/html/body/div[2]/div[4]/div/div/div[3]/div[3]/div[2]/p'

filmes = []
for i in range(51,61):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    details = tree.xpath(xpath_details)
    
    nome = get_element_text(details[0])
    if "(" in nome:
        filme['nome']=nome[:nome.index("(")].strip()
        filme['ano']=nome[nome.index("(")+1:nome.index(")")]
        
    diretor = get_element_text(details[1])
    if diretor.rfind("-")>-1:
        filme['diretor']=diretor[diretor.rfind("-")+1:].strip()
        
    roteiro = get_element_text(details[3])
    if roteiro.find(":") > -1:
        filme[remover_acentos(roteiro[:roteiro.find(":")].lower())] = roteiro[roteiro.find(":")+1:].strip()
        
    elenco = get_element_text(details[4])
    if elenco.find(":") > -1:
        filme[remover_acentos(elenco[:elenco.find(":")].lower())] = elenco[elenco.find(":")+1:].strip()
    
    filmes.append(filme)
filmes

[{'nome': 'O Predador',
  'ano': '2018',
  'diretor': 'Shane Black',
  'roteiro': 'Fred Dekker, Shane Black',
  'elenco': 'Boyd Holbrook, Trevante Rhodes, Jacob Tremblay, Keegan-Michael Key, Olivia Munn, Thomas Jane, Alfie Allen, Sterling K. Brown, Augusto Aguilera, Jake Busey, Yvonne Strahovski, Edward James Olmos, Niall Matter, Dean Redman, Steve Wilder, Nikolas Dukic, Andrew Jenkins, Crystal Mudry, Paul Lazenby, Lochlyn Munro'},
 {'nome': 'Benzinho',
  'ano': '2018',
  'diretor': 'Gustavo Pizzi',
  'roteiro': 'Gustavo Pizzi, Karine Teles',
  'elenco': 'Karine Teles, Otávio Müller, Adriana Esteves, Konstantinos Sarris, César Troncoso, Mateus Solano, Camilo Pellegrini, Ariclenes Barroso, Pablo Riera, Lucas Gouvêa, Ingrid Guimarães'},
 {'nome': 'A Barraca do Beijo',
  'ano': '2018',
  'diretor': 'Vince Marcello',
  'roteiro': 'Vince Marcello',
  'elenco': 'Joey King, Joel Courtney, Jacob Elordi, Molly Ringwald, Carson White, Frances Sholto-Douglas, Nathan Lynn, Meganne Young, Byron Lan

In [15]:
rapadura = [{'nome': 'O Predador',  'ano': '2018',  'diretor': 'Shane Black',  'roteiro': 'Fred Dekker, Shane Black',  'elenco': 'Boyd Holbrook, Trevante Rhodes, Jacob Tremblay, Keegan-Michael Key, Olivia Munn, Thomas Jane, Alfie Allen, Sterling K. Brown, Augusto Aguilera, Jake Busey, Yvonne Strahovski, Edward James Olmos, Niall Matter, Dean Redman, Steve Wilder, Nikolas Dukic, Andrew Jenkins, Crystal Mudry, Paul Lazenby, Lochlyn Munro'}, {'nome': 'Benzinho',  'ano': '2018',  'diretor': 'Gustavo Pizzi',  'roteiro': 'Gustavo Pizzi, Karine Teles',  'elenco': 'Karine Teles, Otávio Müller, Adriana Esteves, Konstantinos Sarris, César Troncoso, Mateus Solano, Camilo Pellegrini, Ariclenes Barroso, Pablo Riera, Lucas Gouvêa, Ingrid Guimarães'}, {'nome': 'A Barraca do Beijo',  'ano': '2018',  'diretor': 'Vince Marcello',  'roteiro': 'Vince Marcello',  'elenco': 'Joey King, Joel Courtney, Jacob Elordi, Molly Ringwald, Carson White, Frances Sholto-Douglas, Nathan Lynn, Meganne Young, Byron Langley, D. David Morin, Dan Elijah Rudin, Robbie Tucker, Jessica Sutton, Michelle Allen, Joshua Daniel Eady, Judd Krok, Chase Dallas, Celina Caroto, Ben Lillie, Megan du Plessis'}, {'nome': 'Meu Ex é um Espião',  'ano': '2018',  'diretor': 'Susanna Fogel',  'roteiro': 'Susanna Fogel, David Iserson',  'elenco': 'Mila Kunis, Kate McKinnon, Justin Theroux, Gillian Anderson, Sam Heughan, Hasan Minhaj, Ivanna Sakhno, Fred Melamed, Jane Curtin, Paul Reiser, James Fleet, Carolyn Pickles, Kev Adams, Lolly Adefope, Dustin Demri-Burns, Justine Wachsberger, Ólafur Darri Ólafsson, Attila Árpa, Ruby Kammer, Genevieve McCarthy'}, {'nome': 'A Freira',  'ano': '2018',  'diretor': 'Corin Hardy',  'roteiro': 'Gary Dauberman',  'elenco': 'Bonnie Aarons, Demián Bichir, Taissa Farmiga, Charlotte Hope, Ingrid Bisu, Manuela Ciucur, Jonas Bloquet, Jonny Coyne, Jared Morgan, Sandra Teles, Boiangiu Alma, August Maturo'}, {'nome': 'Mamma Mia! Lá Vamos Nós de Novo',  'ano': '2018',  'diretor': 'Ol Parker',  'roteiro': 'Ol Parker, Richard Curtis, Catherine Johnson',  'elenco': 'Amanda Seyfried, Lily James, Dominic Cooper, Christine Baranski, Julie Walters, Pierce Brosnan, Colin Firth, Stellan Skarsgård, Andy García, Cher, Meryl Streep, Jessica Keenan Wynn, Alexa Davies, Jeremy Irvine, Hugh Skinner, Josh Dylan, Omid Djalili, Celia Imrie, Maria Vacratsis, Panos Mouzourakis'}, {'nome': 'Onde Está Você, João Gilberto?',  'ano': '2018',  'diretor': 'Georges Gachot',  'roteiro': '',  'elenco': 'João Gilberto'}, {'nome': 'Slender Man: Pesadelo Sem Rosto',  'ano': '2018',  'diretor': 'Sylvain White',  'roteiro': 'David Birke',  'elenco': 'Joey King, Javier Botet, Julia Goldani Telles, Jaz Sinclair, Kevin Chapman, Annalise Basso, Talitha Bateman, Alex Fitzalan, Michael Reilly Burke, Kayla Caulfield, Jessica Blank, Jeffrey Corazzini, Adrian M. Mompoint, Michael Tow'}, {'nome': 'Perfeita Pra Você',  'ano': '2018',  'diretor': 'Stephanie Laing',  'roteiro': 'Bess Wohl',  'elenco': "Gugu Mbatha-Raw, Michiel Huisman, Christopher Walken, Steve Coogan, Jacki Weaver, Claire Holt, Timothy Simons, Gayle Rankin, Kate McKinnon, Tamara Tunie, Jessie Ennis, Brian Tyree Henry, Glenn Fleshler, Zabryna Guevara, Richard Holmes, Sawyer Barth, Celeste O'Connor, Alyssa Cheatham, Zachary Hernandez, Grace Rex"}, {'nome': 'O Paciente: O Caso Tancredo Neves',  'ano': '2018',  'diretor': 'Sergio Rezende',  'roteiro': '',  'elenco': ''}]
filmes == rapadura

True

### Plano Crítico

In [6]:
def text_normalize(text):
    return normalize("NFKD",text)

In [8]:
xpath_details = '(//*[@id="content-area"]/p)[last()]'

filmes = []
for i in range(61,71):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    details = tree.xpath(xpath_details)
    details = get_element_text(details[0]).split("\n")
    filme['nome'] = text_normalize(details[0])
    for item in details[1:]:
        if item.find(":") > -1:
            filme[remover_acentos(item[:item.find(":")].lower())] = text_normalize(item[item.find(":")+1:].strip())
    filmes.append(filme)
filmes

[{'nome': 'Funny Girl – Uma Garota Genial (Funny Girl) – EUA, 1968',
  'direcao': 'William Wyler',
  'roteiro': 'Isobel Lennart (baseado em Funny Girl, escrito por Isobel Lennart, Jule Stynne e Bob Merrill)',
  'elenco': 'Barbra Streisand, Omar Sharif, Kay Medford, Anne Francis, Walter Pidgeon, Lee Allen, Mae Questel, Gerald Mohr, Frank Faylen, Alena Johnston, Thordis Brandt, Arthur Tovey',
  'duracao': '149 min.'},
 {'nome': 'Simplesmente Martha (Bella Martha/Alemanha, 2001)',
  'direcao': 'Sandra Nettelbeck',
  'roteiro': 'Sandra Nettelbeck',
  'elenco': 'Antonio Wannek, August Zirner, Diego Ribon, Idil Üner, Katja Studt, Martina Gedeck, Maxime Foerste, Oliver Broumis, Sergio Castellitto, Sibylle Canonica, Ulrich Thomsen',
  'duracao': '103 min.'},
 {'nome': 'O Mistério do Relógio na Parede (The House with a Clock in Its Walls) EUA, 2018',
  'direcao': 'Eli Roth',
  'roteiro': 'Eric Kripke, John Bellairs',
  'elenco': 'Owen Vaccaro, Jack Black, Cate Blanchett, Kyle MacLachlan, Col

In [14]:
planocritico = [{'nome': 'Funny Girl – Uma Garota Genial (Funny Girl) – EUA, 1968',  'direcao': 'William Wyler',  'roteiro': 'Isobel Lennart (baseado em Funny Girl, escrito por Isobel Lennart, Jule Stynne e Bob Merrill)',  'elenco': 'Barbra Streisand, Omar Sharif, Kay Medford, Anne Francis, Walter Pidgeon, Lee Allen, Mae Questel, Gerald Mohr, Frank Faylen, Alena Johnston, Thordis Brandt, Arthur Tovey',  'duracao': '149 min.'}, {'nome': 'Simplesmente Martha (Bella Martha/Alemanha, 2001)',  'direcao': 'Sandra Nettelbeck',  'roteiro': 'Sandra Nettelbeck',  'elenco': 'Antonio Wannek, August Zirner, Diego Ribon, Idil Üner, Katja Studt, Martina Gedeck, Maxime Foerste, Oliver Broumis, Sergio Castellitto, Sibylle Canonica, Ulrich Thomsen',  'duracao': '103 min.'}, {'nome': 'O Mistério do Relógio na Parede (The House with a Clock in Its Walls) EUA, 2018',  'direcao': 'Eli Roth',  'roteiro': 'Eric Kripke, John Bellairs',  'elenco': 'Owen Vaccaro, Jack Black, Cate Blanchett, Kyle MacLachlan, Collen Camp, Sunny Suljic, Lorenza Izzo, Braxton Bjerken, Vanessa Anne Williams, Charles Green, De’Jon Watts, Aaron Beelner, Joshua Phillips, Christian Calloway',  'duracao': '105 min.'}, {'nome': 'Batman – O Retorno (Batman Returns, EUA/ Reino Unido – 1992)',  'direcao': 'Tim Burton',  'roteiro': 'Daniel Waters (baseado em história dele e de Sam Hamm)',  'elenco': 'Michael Keaton, Danny DeVito, Michelle Pfeiffer, Christopher Walken, Michael Gough, Michael Murphy, Cristi Conaway, Andrew Bryniarski, Pat Hingle, Vincent Schiavelli, Steve Witting, Jan Hooks',  'duracao': '126 min.'}, {'nome': 'Uma Questão Pessoal (Una questione privata) — Itália, França, 2017',  'direcao': 'Paolo Taviani',  'roteiro': 'Paolo Taviani, Vittorio Taviani (baseado na obra de Beppe Fenoglio)',  'elenco': 'Luca Marinelli, Lorenzo Richelmy, Valentina Bellè, Francesca Agostini, Jacopo Olmo Antinori, Antonella Attili, Giulio Beranek, Mario Bois, Marco Brinzi, Fabrizio Costella, Mauro Conte, Lorenzo Demaria, Andrea Di Maria, Guglielmo Favilla, Anna Ferruzzo',  'duracao': '84 min.'}, {'nome': 'A Nova Onda do Imperador (The Emperor’s New Groove) – EUA, 2000',  'direcao': 'Mark Dindal',  'roteiro': 'David Reynolds, Mark Dindal, Chris Williams, Roger Allers',  'elenco': 'David Spade, John Goodman, Eartha Kitt, Patrick Warburton, Wendie Malick, Kellyann Kelso, Eli Russell Linnetz, Stephen J. Anderson, Bob Bergen, Patti Deutsch, John Fiedler, Sherry Lynn, Mickie McGowan',  'duracao': '78 min.'}, {'nome': 'Buscando... (Searching..., Estados Unidos- 2018)',  'direcao': 'Aneesh Chaganty',  'roteiro': 'Aneesh Chaganty, Sev Ohanian',  'elenco': 'Alex Jayne Go, Ashley Edner, Benjamin J. Cain Jr., Brad Abrell, Courtney Lauren Cummings, Debra Messing, Erin Henriques, Gabriel D. Angell, Gage Biltoft, John Cho, Joseph Lee, Kyle Austin Brown, Lasaundra Gibson, Melissa Disney, Michelle La, Morgan Peter Brown, Rasha Goel, Reed Buck, Roy Abramsohn, Sara Sohn, Steven Michael Eich, Thomas Barbusca',  'duracao': '1o1 min'}, {'nome': 'Laranja Mecânica (A Clockwork Orange, Reino Unido/EUA – 1971)',  'direcao': 'Stanley Kubrick',  'roteiro': 'Stanley Kubrick (baseado em romance de Anthony Burgess)',  'elenco': 'Malcolm McDowell, Patrick Magee, Michael Bates, Warren Clarke, John Clive, Adrienne Corri, Carl Duering, Paul Farrell, Clive Francis, Michael Gover, Miriam Karlin, James Marcus, Aubrey Morris, Godfrey Quigley, Sheila Raynor',  'duracao': '136 min.'}, {'nome': 'Adeus Christopher Robin (Goodbye Christopher Robin) – Reino Unido, 2017',  'direcao': 'Simon Curtis',  'roteiro': 'Frank Cottrell Boyce, Simon Vaughan',  'elenco': 'Kelly Macdonald, Margot Robbie, Domnhall Gleeson, Will Tilston, Alex Lawther, Stephen Campbell Moore, Richard McCabe, Geradine Sommerville, Mossie Smith, Vicki Pepperdine',  'duracao': '107 min'}, {'nome': 'Bleach – Japão, 2018',  'direcao': 'Shinsuke Sato',  'roteiro': 'Shinsuke Sato, Daisuke Habara, Tite Kubo',  'elenco': 'Sota Fukushi, Hana Sugisaki, Ryo Yoshizawa, Erina Mano, Yu Koyanagi, Taichi Saotome, Miyavi, Seiichi Tanabe, Masami Nagasawa, Yosuke Eguchi',  'duracao': '108 min.'}]
filmes==planocritico

True

### Plano Aberto

In [9]:
xpath_details = '//*[@id="fica-tecnica"]/div[2]/p'

filmes = []
for i in range(71,81):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    details = tree.xpath(xpath_details)
    
    filme['nome'] = get_element_text(details[0])
    for item in details[1:]:
        item = get_element_text(item)
        if item.find(":") > -1:
            filme[remover_acentos(item[:item.find(":")].lower())] = item[item.find(":")+1:].strip()
    filmes.append(filme)
filmes

[{'nome': 'Bloqueio, 2018 - Brasil',
  'direcao': 'Victória Álvares, Quentin Delaroche',
  'elenco': 'Marocs da Silva dos Santos, Benedito Amauro, Parazinho'},
 {'nome': 'New Life S.A., 2018 – Brasil',
  'direcao': 'André Carvalheira',
  'roteiro': 'André Carvalheira, Aurélio Aragão, Pablo Gonçalo',
  'elenco': 'Fernanda Rocha, Murilo Grossi, Catarina Accioly, Eric Naves, Edu Moares, Juliano Coacci, André Deca, Bianca Terraza, Wellington Abreu, Marcelo Pelucio',
  'fotografia': 'Krishna Schmidt',
  'design de producao': 'Maíra Carvalho'},
 {'nome': 'Excelentíssimos, 2018 - Brasil', 'direcao': 'Douglas Duarte'},
 {'nome': 'Torre das Donzelas, 2018 - Brasil',
  'direcao': 'Susanna Lira',
  'roteiro': 'Susanna Lira'},
 {'nome': 'Domingo, 2018 – Brasil',
  'direcao': 'Fellipe Barbosa, Clara Linhart',
  'roteiro': 'Lucas Paraizo',
  'elenco': 'Itala Nardi, Camila Morgado, Augusto Madeira, Michael Wahrmann, Martha Nowill, Clemente Viscaíno, Ismael Caneppele'},
 {'nome': 'Carnivores, 2018 - F

In [20]:
planoaberto = [{'nome': 'Bloqueio, 2018 - Brasil',  'direcao': 'Victória Álvares, Quentin Delaroche',  'elenco': 'Marocs da Silva dos Santos, Benedito Amauro, Parazinho'}, {'nome': 'New Life S.A., 2018 – Brasil',  'direcao': 'André Carvalheira',  'roteiro': 'André Carvalheira, Aurélio Aragão, Pablo Gonçalo',  'elenco': 'Fernanda Rocha, Murilo Grossi, Catarina Accioly, Eric Naves, Edu Moares, Juliano Coacci, André Deca, Bianca Terraza, Wellington Abreu, Marcelo Pelucio',  'fotografia': 'Krishna Schmidt',  'design de producao': 'Maíra Carvalho'}, {'nome': 'Excelentíssimos, 2018 - Brasil', 'direcao': 'Douglas Duarte'}, {'nome': 'Torre das Donzelas, 2018 - Brasil',  'direcao': 'Susanna Lira',  'roteiro': 'Susanna Lira'}, {'nome': 'Domingo, 2018 – Brasil',  'direcao': 'Fellipe Barbosa, Clara Linhart',  'roteiro': 'Lucas Paraizo',  'elenco': 'Itala Nardi, Camila Morgado, Augusto Madeira, Michael Wahrmann, Martha Nowill, Clemente Viscaíno, Ismael Caneppele'}, {'nome': 'Carnivores, 2018 - França e Bélgica',  'direcao': 'Jérémie Renier & Yannick Renier',  'roteiro': 'Bulle Decarpentries, Jérémie Guez, Yaël Langmann, Jérémie Renier, Yannick Renier & Agnès de Sacy',  'elenco': 'Leïla Bekhti, Zita Hanrot, Bastien Bouillon, Johan Heldenbergh, Hiam Abbass & Octave Bossuet',  'fotografia': 'Georges Lechaptois',  'trilha sonora': 'Avia',  'montagem': 'Nico Leunen',  'design de producao': 'Nora Salhi'}, {'nome': 'The Predator, 2018 - Canadá e Estados Unidos',  'direcao': 'Shane Black',  'roteiro': 'Fred Dekker & Shane Black',  'elenco': 'Boyd Holbrook, Trevante Rhodes, Olivia Munn, Jacob Tremblay, Sterling K. Brown, Keegan-Michael Key, Thomas Jane, Alfie Allen, Augusto Aguilera & Yvonne Strahovski',  'fotografia': 'Larry Fong',  'trilha sonora': 'Henry Jackman',  'montagem': 'Harry B. Miller III',  'design de producao': 'Martin Whist'}, {'nome': 'The Nun, 2018 - Estados Unidos',  'direcao': 'Corin Hardy',  'roteiro': 'Gary Dauberman',  'elenco': 'Taissa Farmiga, Demián Bichir, Jonas Bloquet, Bonnie Aarons, Ingrid Bisu, Charlotte Hope, Sandra Teles, August Maturo, Jack Falk & Lynnette Gaza',  'fotografia': 'Maxime Alexandre',  'trilha sonora': 'Abel Korzeniowski',  'montagem': 'Michel Aller, Ken Blackwell',  'design de producao': 'Jennifer Spence'}, {'nome': 'Searching, 2018 - Estados Unidos',  'direcao': 'Aneesh Chaganty',  'roteiro': 'Aneesh Chaganty & Sev Ohanian',  'elenco': 'John Cho, Sara Sohn, Michelle La, Joseph Lee & Debra Messing',  'fotografia': 'Juan Sebastian Baron, Nicholas D. Johnson & Will Merrick',  'trilha sonora': 'Torin Borrowdale',  'montagem': 'Nicholas D. Johnson, Will Merrick',  'design de producao': 'Angel Herrera'}, {'nome': 'O Nome da Morte, 2017 - Brasil',  'direcao': 'Henrique Goldman',  'roteiro': 'Henrique Goldman, George Moura',  'elenco': 'Marco Pigossi, Fabíula Nascimento, André Mattos, Matheus Nachtergaele, Tony Tornado, Augusto Madeira, Martha Nowill',  'fotografia': 'Azul Serra',  'trilha sonora': 'Brian Eno',  'montagem': 'Lívia Serpa',  'design de producao': 'Tiago Marques'}]
filmes == planoaberto

True

### Cinema Sim

In [21]:
xpath_details = '//*[@id="site-top"]/div[1]/div[4]/div/div/main/article/div[1]/div[1]/div[1]/div[2]'
portuguese_name = '//*[@id="site-top"]/div[1]/div[3]/div[2]/div/h1'

filmes = []
for i in range(81,91):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    
    nome = tree.xpath(portuguese_name)[0].text
    if nome.find("|") > -1:
        filme['nome'] = nome[:nome.find("|")].strip()
    else:
        filme['nome'] = nome
    
    details = tree.xpath(xpath_details)
    details = details[0].text.split('\n')
    for item in details:
        if item.find(":") > -1:
            filme[remover_acentos(item[:item.find(":")].lower())] = item[item.find(":")+1:].strip()
        
    filmes.append(filme)
filmes

[{'nome': 'O Retorno do Herói',
  'titulo original': 'Le Retour du Héros',
  'lancamento': '20 de setembro',
  'direcao': 'Laurent Tirard',
  'roteiro': 'Laurent Tirard, Grégoire Vigneron',
  'genero': 'Comédia',
  'elenco': 'Jean Dujardin, Mélanie Laurent, Noémie Merlant',
  'nacionalidade': 'França, Bélgica'},
 {'nome': 'Marvin',
  'titulo original': 'Marvin ou La Belle Éducation',
  'lancamento': '06 de setembro',
  'direcao': 'Anne Fontaine',
  'roteiro': 'Anne Fontaine, Pierre Trividic',
  'genero': 'Drama',
  'elenco': 'Finnegan Oldfield, Grégory Gadebois, Vincent Macaigne',
  'nacionalidade': 'França'},
 {'nome': 'Sierra Burgess é uma Loser',
  'titulo original': 'Sierra Burgess is Loser',
  'lancamento': '07 de setembro de 2018',
  'direcao': 'Ian Samuels',
  'roteiro': 'Lindsey Beer',
  'genero': 'Comédia Romântica',
  'elenco': 'Shannon Purser, RJ Cyler, Noah Centineo, Will Peltz, Kristine Froseth, Lea Thompson e Alan Ruck',
  'nacionalidade': 'Estados Unidos'},
 {'nome': 'Cr

In [22]:
cinemasim = [{'nome': 'O Retorno do Herói',  'titulo original': 'Le Retour du Héros',  'lancamento': '20 de setembro',  'direcao': 'Laurent Tirard',  'roteiro': 'Laurent Tirard, Grégoire Vigneron',  'genero': 'Comédia',  'elenco': 'Jean Dujardin, Mélanie Laurent, Noémie Merlant',  'nacionalidade': 'França, Bélgica'}, {'nome': 'Marvin',  'titulo original': 'Marvin ou La Belle Éducation',  'lancamento': '06 de setembro',  'direcao': 'Anne Fontaine',  'roteiro': 'Anne Fontaine, Pierre Trividic',  'genero': 'Drama',  'elenco': 'Finnegan Oldfield, Grégory Gadebois, Vincent Macaigne',  'nacionalidade': 'França'}, {'nome': 'Sierra Burgess é uma Loser',  'titulo original': 'Sierra Burgess is Loser',  'lancamento': '07 de setembro de 2018',  'direcao': 'Ian Samuels',  'roteiro': 'Lindsey Beer',  'genero': 'Comédia Romântica',  'elenco': 'Shannon Purser, RJ Cyler, Noah Centineo, Will Peltz, Kristine Froseth, Lea Thompson e Alan Ruck',  'nacionalidade': 'Estados Unidos'}, {'nome': 'Crô em Família',  'titulo original': 'Crô em Família',  'ano': '2018',  'direcao': 'Cininha de Paula',  'roteiro': 'Leandro Soares',  'genero': 'Comédia',  'elenco': 'Marcelo Serrado, Jefferson Schroeder, Karina Marthin, João Baldasserini, Mel Maia, Mary Sheyla, Fabiana Karla, Pabllo Vittar, Jojo Todynho, Carol Sampaio, Preta Gil, Gigante Leo, Luis Miranda, Marcus Majella e Marcos Caruso.'}, {'nome': 'Candidato Honesto 2 – Sério?',  'titulo original': 'Candidato Honesto 2',  'lancamento': '30 de agosto',  'direcao': 'Roberto Santucci',  'roteiro': 'Paulo Cursino',  'genero': 'Comédia',  'elenco': 'Leandro Hassum, Flávia Garrafa, Rosanne Mulholland, Victor Leal, Cassio Pandolfh, Maria Padilha.'}, {'nome': 'Os Jovens Titãs em Ação! Nos Cinemas',  'titulo original': 'Teen Titans Go! To The Movies',  'ano': '2018',  'direcao': 'Peter Rida Michail, Aaron Horvath',  'roteiro': 'Aaron Horvath, Michael Jelenic',  'genero': 'Animação',  'elenco': 'Kristen Bell, Will Arnett, Hynden Walch, Tara Strong, Khary Payton, Scott Menville, Greg Cipes'}, {'nome': 'Para Todos Os Garotos Que Já Amei',  'direcao': 'Susan Johnson',  'elenco': 'Lana Condor, John Corbett, Noah Centineo mais'}, {'nome': 'As Herdeiras',  'titulo original': 'Las herederas',  'lancamento': '30 de Agosto',  'direcao': 'Marcelo Martinessi',  'genero': 'Drama',  'elenco': 'Ana Brun, Margarita Irun, Ana Ivanova.'}, {'nome': 'Mamma Mia! Lá Vamos Nós de Novo',  'titulo original': 'Mamma Mia! Here We Go Again',  'lancamento': '02 de agosto de 2018',  'direcao': 'Ol Parker',  'roteiro': 'Ol Parker',  'genero': 'Musical',  'elenco': 'Pierce Brosnan, Meryl Streep, Dominic Cooper, Lily James, Andy García, Amanda Seyfried, Jeremy Irvine, Colin Firth, Hugh Skinner, Stellan Skarsgård, Josh Dylan, Christine Baranski, Jessica Keenan Wynn, Julie Walters, Alexa Davies, Cher.'}, {'nome': 'Gauguin: Viagem ao Taiti',  'titulo original': 'Gauguin: Voyage de Tahiti',  'lancamento': '23 de agosto',  'direcao': 'Edouard Deluc',  'roteiro': 'Edouard Deluc, Etienne Comar, Thomas Lilti, Sarah Kaminsky',  'genero': 'Drama, Biografia',  'elenco': 'Vincent Cassel, Tuheï Adams, Malik Zidi, Pernille Bergendorff',  'nacionalidade': 'França'}]
filmes == cinemasim

True

### Pocilga

In [48]:
xpath_details = '//*[starts-with(@id, "post-")]/div[2]/blockquote[2]/p'
xpath_title = '//*[starts-with(@id, "post-")]/div[2]/h4'

filmes = []
for i in range(91,101):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    title = tree.xpath(xpath_title)
    title = get_element_text(title[0])
    filme["nome"] = title.strip()
    details = tree.xpath(xpath_details)
    details = get_element_text(details[0]).split("\n")
    for item in details:
        if item.find(":") > -1:
            filme[remover_acentos(item[:item.find(":")].lower())] = item[item.find(":")+1:].strip()
    filmes.append(filme)
    
filmes

[{'nome': 'O Mistério do Relógio na Parede (The House with a Clock in Its Walls)',
  'direcao': 'Eli Roth',
  'roteiro': 'Eric Kripke',
  'elenco': 'Jack Black, Cate Blanchett, Owen Vaccaro, Renée Elise Goldsberry, Sunny Suljic e Kyle MacLachlan',
  'genero': 'Comédia, Fantasia, Família',
  'ano': '2018',
  'duracao': '105 minutos'},
 {'nome': 'Buscando… (Searching)',
  'direcao': 'Aneesh Chaganty',
  'roteiro': 'Aneesh Chaganty e Sev Ohanian',
  'elenco': 'John Cho, Michelle La, Debra Messing, Sara Sohn e Joseph Lee',
  'genero': 'Drama, Mistério, Thriller',
  'ano': '2018',
  'duracao': '102 minutos'},
 {'nome': 'Crimes em Happytime (The Happytime Murders)',
  'direcao': 'Brian Henson',
  'roteiro': 'Todd Berger',
  'elenco': 'Melissa McCarthy, Maya Rudolph, Joel McHale, Elizabeth Banks, Leslie David Baker, Bill Barretta e Dorien Davies',
  'genero': 'Ação, Comédia, Crime',
  'ano': '2018',
  'duracao': '91 minutos'},
 {'nome': '10 Segundos para Vencer',
  'direcao': 'José Alvarenga 

In [52]:
pocilga = [{'nome': 'O Mistério do Relógio na Parede (The House with a Clock in Its Walls)',  'direcao': 'Eli Roth',  'roteiro': 'Eric Kripke',  'elenco': 'Jack Black, Cate Blanchett, Owen Vaccaro, Renée Elise Goldsberry, Sunny Suljic e Kyle MacLachlan',  'genero': 'Comédia, Fantasia, Família',  'ano': '2018',  'duracao': '105 minutos'}, {'nome': 'Buscando… (Searching)',  'direcao': 'Aneesh Chaganty',  'roteiro': 'Aneesh Chaganty e Sev Ohanian',  'elenco': 'John Cho, Michelle La, Debra Messing, Sara Sohn e Joseph Lee',  'genero': 'Drama, Mistério, Thriller',  'ano': '2018',  'duracao': '102 minutos'}, {'nome': 'Crimes em Happytime (The Happytime Murders)',  'direcao': 'Brian Henson',  'roteiro': 'Todd Berger',  'elenco': 'Melissa McCarthy, Maya Rudolph, Joel McHale, Elizabeth Banks, Leslie David Baker, Bill Barretta e Dorien Davies',  'genero': 'Ação, Comédia, Crime',  'ano': '2018',  'duracao': '91 minutos'}, {'nome': '10 Segundos para Vencer',  'direcao': 'José Alvarenga Jr.',  'roteiro': 'Thomas Stavros',  'elenco': 'Daniel de Oliveira, Osmar Prado, Ricardo Gelli, Sandra Corveloni, Rafael Andrade, Samuel Toledo e Keli Freitas',  'genero': 'Drama, Esporte, Biografia',  'ano': '2018',  'duracao': '120 minutos'}, {'nome': 'A Primeira Noite de Crime (The First Purge)',  'direcao': 'Gerard McMurray',  'roteiro': 'James DeMonaco',  'elenco': 'Y’lan Noel, Lex Scott Davis, Joivan Wade, Mugga, Patch Darragh e Marisa Tomei',  'genero': 'Ação, Terror, Ficção científica e Thriller',  'ano': '2018',  'duracao': '98 minutos'}, {'nome': 'Um Pequeno Favor (A Simple Favor)',  'direcao': 'Paul Feig',  'roteiro': 'Jessica Sharzer',  'elenco': 'Anna Kendrick, Blake Lively, Henry Golding e Andrew Rannells',  'genero': 'Comédia, Crime, Drama',  'ano': '2018',  'duracao': '117 minutos'}, {'nome': 'O Paciente – O Caso Tancredo Neves',  'direcao': 'Sérgio Rezende',  'roteiro': 'Gustavo Liptzein',  'elenco': 'Othon Bastos, Esther Góes, Leonardo Medeiros, Otávio Müller, Paulo Betti, Emílio Dantas, Mário Hermeto e Lucas Drummond',  'genero': 'Biografia',  'ano': '2018',  'duracao': '100 minutos'}, {'nome': 'O Predador (The Predator)',  'direcao': 'Shane Black',  'roteiro': 'Fred Dekker e Shane Black',  'elenco': 'Boyd Holbrook, Trevante Rhodes, Jacob Tremblay, Keegan-Michael Key, Olivia Munn, Thomas Jane, Alfie Allen e Sterling K. Brown',  'genero': 'Ação, Aventura, Terror',  'ano': '2018',  'duracao': '107 minutos'}, {'nome': 'O Banquete',  'direcao': 'Daniela Thomas',  'roteiro': 'Daniela Thomas',  'elenco': 'Drica Moraes, Mariana Lima, Caco Ciocler, Rodrigo Bolzan, Fabiana Gugli, Gustavo Machado, Bruna Linzmeyer, Chay Suede e Georgette Fedal',  'genero': 'Suspense, Drama',  'ano': '2018',  'duracao': '104 minutos'}, {'nome': 'Christopher Robin: Um Reencontro Inesquecível (Christopher Robin)',  'direcao': 'Marc Forster',  'roteiro': 'Alex Ross Perry, Tom McCarthy e Allison Schroeder; história de Greg Brooker e Mark Steven Johnson',  'elenco': 'Ewan McGregor, Hayley Atwell, Jim Cummings e Brad Garrett',  'genero': 'Animação, Aventura, Comédia',  'ano': '2018',  'duracao': '104 minutos'}]
filmes == pocilga

True